# CFB Week 10 Methods Testing

### <font color='orange'>UPDATE COLLEGE FOOTBALL WEEK NUMBER HERE</font>
Cell below contains week number and year for filtering API data. 

sat_day_num is used for getting the day of the month for this week's football games since don't want to bet on Thursday or Friday games

df_name_prefix is how the file names for exported dataframes will be stored as so it is easy to know what I am looking at before opening and for sorting later

In [1]:
year_num = 2022
week_num = 10
sat_day_num = 5
df_name_prefix = 'cfb.' + str(year_num) + '.' + str(week_num)

### Import libraries

In [2]:
import pandas as pd
import requests
import json
import os
import numpy as np
from bs4 import BeautifulSoup
import config

In [3]:
# cfbd imports
from __future__ import print_function
import time
import cfbd
from cfbd.rest import ApiException
from pprint import pprint

In [4]:
# import custom formulas
from cfbd_transform import betting_api_dataframe
from equations import parlay_multiplier, amer_odds_to_prob, amer_odds_to_decimal

### Set up environment

In [5]:
# set env variable. need to fill in own
os.environ['api_key'] = config.api_key

In [6]:
# Configure API key authorization: ApiKeyAuth
configuration = cfbd.Configuration()
configuration.api_key['Authorization'] = os.getenv('api_key')
configuration.api_key_prefix['Authorization'] = 'Bearer'

## Import and Transform Data

### Import Rankings

In [7]:
# getting FBS ranking
rankings_api = cfbd.RankingsApi(cfbd.ApiClient(configuration))
rankings_api_response = rankings_api.get_rankings(year = year_num, \
                                                  week = week_num, \
                                                  season_type='regular')

# save result into dataframe/ convert from json
rankings_df = pd.DataFrame.from_records([t.to_dict() for t in rankings_api_response])

# save as dataframe in better formatting
rankings_df = pd.DataFrame.from_dict(rankings_df['polls'][0])

# excluding polls for FCS and levels below that
poll_exclude = ['FCS Coaches Poll','AFCA Division II Coaches Poll','AFCA Division III Coaches Poll']
rankings_df = rankings_df[~rankings_df['poll'].isin(poll_exclude)]

# coaches poll
poll = 'Coaches Poll'
index_num = rankings_df[rankings_df['poll']==poll].index[0]
coaches_poll_df = pd.DataFrame(rankings_df[rankings_df['poll']==poll]['ranks'][index_num])
coaches_poll_df['poll_name'] = poll

# AP Top 25 poll
poll = 'AP Top 25'
index_num = rankings_df[rankings_df['poll']==poll].index[0]
ap_poll_df = pd.DataFrame(rankings_df[rankings_df['poll']==poll]['ranks'][index_num])
ap_poll_df['poll_name'] = poll

### Import Team Records

In [8]:
# getting FBS ranking
rankings_api = cfbd.RankingsApi(cfbd.ApiClient(configuration))
rankings_api_response = rankings_api.get_rankings(year = year_num, \
                                                  week = week_num, \
                                                  season_type='regular')

In [9]:
# get team win loss records
games_api = cfbd.GamesApi(cfbd.ApiClient(configuration))
games_api_response = games_api.get_team_records(year = year_num)

team_record_df = pd.DataFrame.from_records([g.to_dict() for g in games_api_response])

In [10]:
# parse toal win loss column
win_loss = []
for i in team_record_df['total'].apply(pd.Series).columns.values:
    win_loss.append('total_'+i)
    
total_win_loss_df = team_record_df['total'].apply(pd.Series)
total_win_loss_df.columns = win_loss

# append to original dataframe
team_record_df = team_record_df.merge(total_win_loss_df, left_index=True, right_index=True).drop(columns=['total'])

In [11]:
# make a W-L text field
team_record_df['win_loss'] = team_record_df['total_wins'].astype(str) + '-' + \
    team_record_df['total_losses'].astype(str) + '-' + team_record_df['total_ties'].astype(str)

In [12]:
# home win %
home_win_perc = team_record_df['home_games'].apply(pd.Series)[['games','wins']]
home_win_perc['home_win_%'] = round((home_win_perc['wins'] / home_win_perc['games'])*100, 2)

# append to original dataframe
team_record_df = team_record_df.merge(home_win_perc['home_win_%'], left_index=True, right_index=True)#.drop(columns=['home_games'])


In [13]:
# road win %
away_win_perc = team_record_df['away_games'].apply(pd.Series)[['games','wins']]
away_win_perc['away_win_%'] = round((away_win_perc['wins'] / away_win_perc['games'])*100, 2)

# append to original dataframe
team_record_df = team_record_df.merge(away_win_perc['away_win_%'], left_index=True, right_index=True)#.drop(columns=['away_games'])


In [14]:
col_list = ['team', 'conference','win_loss','home_win_%','away_win_%','expected_wins']
team_record_df = team_record_df[col_list]

In [15]:
# add ap poll inf to team record dataset
team_record_df_v2 = pd.merge(team_record_df, ap_poll_df[['school', 'rank', 'first_place_votes', 'points']], \
         how = 'left', left_on = 'team', right_on='school')
team_record_df_v2 = team_record_df_v2.rename(columns = {'rank': 'rank_ap', 
             'first_place_votes': 'first_place_votes_ap', 
             'points': 'points_ap'})
team_record_df_v2.drop(columns='school', inplace = True)

In [16]:
# add coaches poll info to team record dataset
team_record_df_v3 = pd.merge(team_record_df_v2, coaches_poll_df[['school', 'rank', 'first_place_votes', 'points']], \
         how = 'left', left_on = 'team', right_on='school')
team_record_df_v3 = team_record_df_v3.rename(columns = {'rank': 'rank_coaches', 
             'first_place_votes': 'first_place_votes_coaches', 
             'points': 'points_coaches'})
team_record_df_v3.drop(columns='school', inplace = True)

In [17]:
team_record_df_v4 = team_record_df_v3.fillna('')

### Import ELO

In [18]:
ratings_api = cfbd.RatingsApi(cfbd.ApiClient(configuration))

elo_api_response = ratings_api.get_elo_ratings(year = year_num)

In [19]:
elo_df = pd.DataFrame.from_records([t.to_dict() for t in elo_api_response])

In [20]:
elo_df.head()

,year,team,conference,elo
0,2022,Air Force,Mountain West,1744.0
1,2022,Akron,Mid-American,973.0
2,2022,Alabama,SEC,2185.0
3,2022,Appalachian State,Sun Belt,1647.0
4,2022,Arizona,Pac-12,1332.0


### Import Week 8 betting information and transform data

1) Create an instance of betting API
2) Create a start time column that parses the start date column
2) Filter for Sat games only if needed
3) Filter out Washington schools for now (change if in Vegas)
4) Add win loss
5) Add media info to see where to watch
6) Add poll info
7) Add in conference marker
8) Merge ELO ratings

In [21]:
# use cfbd_transform function
betting_df = betting_api_dataframe(configuration = configuration, \
                     week = week_num, \
                     year = year_num)

In [22]:
# create a start_time column
betting_df['start_time'] = betting_df['start_date'].dt.time

In [23]:
# for this week, only want to look at Saturday games
betting_df = betting_df[betting_df['start_date'].dt.day==sat_day_num]

In [24]:
# filter out Washington schools
teams_df = pd.read_csv('./datasets/fbs_teams.csv')

# save school names in WA
wa_schools_list = teams_df[teams_df['state']=='WA']['school']

# filter out WA schools
betting_df = betting_df[(~betting_df['home_team'].isin(wa_schools_list)) & \
                         (~betting_df['away_team'].isin(wa_schools_list))]

In [25]:
# add W-L info to betting dataframe

# first do home team
# rename win_loss to win_loss_home

# drop team column
home_team_cols = ['team','win_loss','home_win_%']
betting_df = pd.merge(betting_df, team_record_df_v4[home_team_cols], \
                      how = 'left',\
                      left_on = 'home_team',\
                      right_on='team').drop(columns=['team']).rename(columns={'win_loss':'win_loss_home'})



# then do away team
away_team_cols = ['team','win_loss','away_win_%']
betting_df = pd.merge(betting_df, team_record_df_v4[away_team_cols], \
                      how = 'left', \
                      left_on = 'away_team', \
                      right_on='team').drop(columns=['team']).rename(columns={'win_loss':'win_loss_away'})

In [26]:
# get media information for this week
media = games_api.get_game_media(year = year_num, week = week_num, classification = 'fbs')
media_df = pd.DataFrame.from_records([i.to_dict() for i in media])
# join with betting dataframe
betting_df = pd.merge(betting_df, media_df[['id','outlet']], how='left', left_on ='id', right_on='id')

In [27]:
# add poll info to betting dataframe
# first do home team

# ap poll
home_team_cols = ['school', 'rank']
betting_df = pd.merge(betting_df, ap_poll_df[home_team_cols], \
         how = 'left',\
         left_on = 'home_team',\
         right_on='school').drop(columns=['school']).rename(columns={'rank':'ap_rank_home'}).fillna('')

# coaches poll
home_team_cols = ['school', 'rank']
betting_df = pd.merge(betting_df, coaches_poll_df[home_team_cols], \
         how = 'left',\
         left_on = 'home_team',\
         right_on='school').drop(columns=['school']).rename(columns={'rank':'coaches_rank_home'}).fillna('')

In [28]:
# add poll info to betting dataframe
# now away team

# ap poll
away_team_cols = ['school', 'rank']
betting_df = pd.merge(betting_df, ap_poll_df[away_team_cols], \
         how = 'left',\
         left_on = 'away_team',\
         right_on='school').drop(columns=['school']).rename(columns={'rank':'ap_rank_away'}).fillna('')

# coaches poll
home_team_cols = ['school', 'rank']
betting_df = pd.merge(betting_df, coaches_poll_df[away_team_cols], \
         how = 'left',\
         left_on = 'away_team',\
         right_on='school').drop(columns=['school']).rename(columns={'rank':'coaches_rank_away'}).fillna('')

In [29]:
'''
values:
1 = in conference game
0 = out of conference game
Nan = two independent schools are playing
'''

def in_conference (df):
    if (df['home_conference'] == 'FBS Independents') & (df['away_conference'] == 'FBS Independents'):
        return np.nan
    elif df['home_conference'] == df['away_conference']:
        return 1
    else:
        return 0
betting_df['is_in_conference'] = betting_df.apply(in_conference, axis=1)

In [30]:
# add elo ratings for away teams
betting_df = pd.merge(betting_df, elo_df[['team','elo']], left_on='away_team', right_on='team', how='left')
betting_df = betting_df.rename(columns={'elo':'elo_away'})

In [31]:
# add elo for home teams
betting_df = pd.merge(betting_df, elo_df[['team','elo']], left_on='home_team', right_on='team', how='left')
betting_df = betting_df.rename(columns={'elo':'elo_home'})

In [32]:
# functions for calculating win probability from elo
def get_away_elo(row):
    exp = (row['elo_home'] - row['elo_away']) / 400
    return 1 / (1 + 10**exp)

def get_home_elo(row):
    exp = (row['elo_away'] - row['elo_home']) / 400
    return 1 / (1 + 10**exp)

In [33]:
elo_cols = ['home_team','away_team']
betting_df['away_elo_prob'] = betting_df.apply(get_away_elo, axis = 1)
betting_df['home_elo_prob'] = betting_df.apply(get_home_elo, axis = 1)

### Export raw transformed data

In [34]:
# export dataframe to csv
betting_df.to_csv('./output/' + df_name_prefix + '.raw.csv', index = False)

## Bet Maker
<b>Process:</b> 

*Note: Only moneylines will be used in parlay. Spread and O/U are more unpredictable in my opinion. 
1. Get games where the home team is an underdog, but the spread is less than 7.5 (can change this cutoff later if needed). Don't want a massive underdog with no chance of winning. For example, if Georgia plays @ Ohio, it is very unlikely and is not a good candidate to include in the parlay / I would never bet moneyline for that.
    - Home field advantage is important in CFB, and I think it is a bigger influence on the outcome of the game than oddsmakers are including in their models
2. Save the initial home underdog dataframe as a csv. Upload to Google Drive
3. Review the initial home underdog output and make cuts. Do this by adding the home team name to the list in the cell below
4. Output the new dataframe as a CSV and upload to Google Drive. Can do a little more digging for matchups and also find games to exclude. If need to exlcude repeat step 3 and add it to the cut list
    - Example: Houston @ Navy 10/22/2022: Didn't like that one since I was sure Houston was going to win. Never sure how a team will play against triple option since not a lot of datapoints for it
4. Input

### Casesars & ESPN Scraping
Try to add caesars odds by scraping:
- ESPN url format for games using game ID: https://www.espn.com/college-football/game?gameId=401411146
- Moneyline is only available pre-game. Spread and O/U still available postgame
- Can also pull in other information like matchup predictor (win prob %), numberFire, SPread cosensu pick, teamrankings, ATS

Steps:
1. Get game id's from cfdb data
2. use game id's to make a list of urls
3. for loop through list to get betting information from espn and win %

In [35]:
# get game IDs from betting_df
game_id_list = betting_df['id'].to_list()

In [36]:
# get urls using game id's
url_list= []
for i in game_id_list:
    url_list.append('https://www.espn.com/college-football/game?gameId=' \
                    + str(i))

In [37]:
# create lists to store home spread, home moneyline, away spread, and away moneyline
home_spread_list = []
home_moneyline_list = []
away_spread_list = []
away_moneyline_list = []
home_espn_win_perc_list = []
away_espn_win_perc_list = []

for i in url_list:    
    # create beautiful soup object for i
    page = requests.get(i)
    soup = BeautifulSoup(page.content, "html.parser")
    # use the beautiful soup object to parse
    results = soup.find(id='gamepackage-pick-center')
    # find home team data and add to lists
    # home team data
    home_data = results.find_all('tr', class_='hometeam')
    for i in home_data:
        for count, x in enumerate(i.find_all('td', class_='score')):
            if count == 2:
                if (x.text.strip().replace(',','').replace('+','') == '--'):
                    home_spread = np.nan
                    home_spread_list.append(np.nan)
                elif x.text.strip().replace(',','').replace('+','') == 'EVEN':
                    home_spread = 0
                    home_spread_list.append(home_spread)
                else:
                    home_spread = float(x.text.strip().replace(',','').replace('+',''))
                    home_spread_list.append(home_spread)
            if count == 3:
                if x.text.strip().replace(',','').replace('+','') == '--':
                    home_moneyline_list.append(np.nan)
                else:
                    home_moneylne = int(x.text.strip().replace(',','').replace('+',''))
                    home_moneyline_list.append(home_moneylne)
        
        home_espn_win_perc_list.append(float(soup.find('span','value-home').text.replace('%', ''))/100)
            
    # find away team data and add to lists
    # away content has the consensus picker data, so need to increase list indexes by 1
    away_data = results.find_all('tr', class_='awayteam')
    for i in away_data:
        for count, x in enumerate(i.find_all('td', class_='score')):
            if count == 3:
                if x.text.strip().replace(',','').replace('+','') == '--':
                    away_spread_list.append(np.nan)
                elif x.text.strip().replace(',','').replace('+','') == 'EVEN':
                    away_spread_list.append(0)
                else:
                    away_spread = float(x.text.strip().replace(',','').replace('+',''))
                    away_spread_list.append(away_spread)            
            if count == 4:
                if x.text.strip().replace(',','').replace('+','') == '--':
                    away_moneyline_list.append(np.nan)
                else:
                    away_moneylne = int(x.text.strip().replace(',','').replace('+',''))
                    away_moneyline_list.append(away_moneylne)
        away_espn_win_perc_list.append(float(soup.find('span','value-away').text.replace('%', ''))/100)

In [38]:
caesars_df = pd.DataFrame(list(zip(game_id_list, home_spread_list, home_moneyline_list, away_spread_list, \
                                   away_moneyline_list, home_espn_win_perc_list, away_espn_win_perc_list)), \
             columns = ['game_id','home_spread_caesars','home_moneyline_caesars',\
                        'away_spread_caesars','away_moneyline_caesars','home_win_prob_espn','away_win_prob_espn'])

In [39]:
caesars_df['home_moneyline_caesars'] = caesars_df['home_moneyline_caesars'].fillna(0)
caesars_df['away_moneyline_caesars'] = caesars_df['away_moneyline_caesars'].fillna(0)

caesars_df['home_win_prob_caesars'] = caesars_df['home_moneyline_caesars'].apply(amer_odds_to_prob).round(7)
caesars_df['away_win_prob_caesars'] = caesars_df['away_moneyline_caesars'].apply(amer_odds_to_prob).round(7)


#caesars_df['away_win_prob_no_vig']

# caesars vig
caesars_df['vig_caesars'] = caesars_df['home_win_prob_caesars'] + caesars_df['away_win_prob_caesars'] - 1

# implied probabilities less vig see:
# https://www.actionnetwork.com/education/juice
# Team A Implied Probability / (Team A IP + Team B IP)
caesars_df['home_win_prob_no_vig_caesars'] = caesars_df['home_win_prob_caesars'] / (caesars_df['home_win_prob_caesars'] + \
                                                                caesars_df['away_win_prob_caesars'])
caesars_df['away_win_prob_no_vig_caesars'] = caesars_df['away_win_prob_caesars'] / (caesars_df['away_win_prob_caesars'] + \
                                                                caesars_df['home_win_prob_caesars'])

In [40]:
betting_df = pd.merge(betting_df, caesars_df, left_on='id', right_on='game_id', how='left')

Get final probabilities and payouts:
- If any in Caesars are blank or nan, use Bovada
- If Bovada also blank or nan, exlude

In [41]:
caesars_columns = ['id', 'home_moneyline_caesars', 'away_moneyline_caesars', 'home_spread_caesars']
betting_df[caesars_columns]

nan_spreads_caesars = betting_df[betting_df['home_spread_caesars'].isnull()].index.to_list()

In [42]:
betting_df['home_moneyline_final'] = np.NaN

In [43]:
# create final moneyline and spread columns that first look at caesars values, and if none use bovada
betting_df['home_moneyline_final'] = betting_df['home_moneyline_caesars'] 
betting_df['away_moneyline_final'] = betting_df['away_moneyline_caesars'] 
betting_df['home_spread_final'] = betting_df['home_spread_caesars'] 

# replace moneyline of 0 with NaN. This handles no moneyline at Caesars
# then use fill na to get moneyline from bovada in the cells where there are nulls
# all moneylines should be filled. if both books don't have a value, the it will remain null
betting_df['home_moneyline_final'] = betting_df['home_moneyline_final'].replace(0, np.nan)
betting_df['home_moneyline_final'] = betting_df['home_moneyline_final'].fillna(betting_df['home_moneyline'])

# repeat for away
betting_df['away_moneyline_final'] = betting_df['away_moneyline_final'].replace(0, np.nan)
betting_df['away_moneyline_final'] = betting_df['away_moneyline_final'].fillna(betting_df['away_moneyline'])

# do spread
betting_df['home_spread_final'] = betting_df['home_spread_final'].fillna(betting_df['spread'])

In [44]:
# calculate win probs based on moneyline odds
betting_df['home_moneyline_final_temp'] = betting_df['home_moneyline_final'].fillna(0)
betting_df['away_moneyline_final_temp'] = betting_df['away_moneyline_final'].fillna(0)

betting_df['home_win_prob_final'] = betting_df['home_moneyline_final_temp'].apply(amer_odds_to_prob).round(7)
betting_df['away_win_prob_final'] = betting_df['away_moneyline_final_temp'].apply(amer_odds_to_prob).round(7)


#caesars_df['away_win_prob_no_vig']

# caesars vig
betting_df['vig_final'] = betting_df['home_win_prob_final'] + betting_df['away_win_prob_final'] - 1

# implied probabilities less vig see:
# https://www.actionnetwork.com/education/juice
# Team A Implied Probability / (Team A IP + Team B IP)
betting_df['home_win_prob_no_vig_final'] = betting_df['home_win_prob_final'] / (betting_df['home_win_prob_final'] + \
                                                                betting_df['away_win_prob_caesars'])
betting_df['away_win_prob_no_vig_final'] = betting_df['away_win_prob_final'] / (betting_df['away_win_prob_final'] + \
                                                                betting_df['home_win_prob_final'])


betting_df.drop(columns=['home_moneyline_final_temp', 'away_moneyline_final_temp'], axis = 1, inplace=True)

### Incorporate SP+ Data
https://www.sbnation.com/college-football/2017/10/13/16457830/college-football-advanced-stats-analytics-rankings

Google Sheets link with SP+ data: https://docs.google.com/spreadsheets/d/1llrN8luL0XWuP8Y-Pb1NXKU84JhXLeUPafy1RfITEDw/edit#gid=1482720576


This link has SP+ and other models:
https://www.ourdailybears.com/baylor-bears-football/2022/8/1/23271366/predictive-statistics-in-cfb-primer


Note: SP+ has some different naming conventions. See cell below for differences

- Miami
    - College Football Data = Miami
    - SP+ = Miami-FL
- Southern Miss
    - College Football Data = Southern Miss
    - SP+ = Southern Miss
- Louisiana Monroe
    - College Football Data = Louisiana Monroe
    - SP+ = UL-Monroe
- Louisiana
    - College Football Data = Louisiana
    - SP+ =  UL-Lafayette
- San José State
    - College Football Data = San José State
    - SP+ = San Jose State
- UMass
    - College Football Data = UMass
    - SP+ = Massachusetts
- South Florida
    - College Football Data = South Florida
    - SP+ = USF
- UT San Antonio
    - College Football Data = UT San Antonio
    - SP+ = UTSA
- Hawai'i
    - College Football Data = Hawai'i
    - SP+ = Hawaii

In [45]:
sp_plus_df = pd.read_csv('./input/SP+/2022 SP+ - Week 10 FBS.csv')

# some values have ' vs. ' some have ' at '. replace vs. with at since had issues with doing two replaces
sp_plus_df['Game_v2'] = sp_plus_df['Game']
sp_plus_df['Game_v2'] = sp_plus_df['Game_v2'].str.replace(' vs. ', ' at ', regex = False)
sp_plus_df['Game_v2'] = sp_plus_df['Game_v2'].str.split(' at ')

# put home and away into own columns
sp_plus_df[['away_team_sp','home_team_sp']] = pd.DataFrame(sp_plus_df['Game_v2'].tolist(), index= sp_plus_df.index)

# convert win probability to float
sp_plus_df['win_prob_float'] = pd.to_numeric(sp_plus_df['Win prob.'].str.strip('%'))


# get home win probabilities based on the 'Proj. winner' column and win probability column
home_win_prob = []
for count, i in enumerate(sp_plus_df['home_team_sp'] == sp_plus_df['Proj. winner']):
    if i == True:
        home_win_prob.append(sp_plus_df.loc[count,'win_prob_float']/100)
    elif i == False:
        home_win_prob.append((100 - sp_plus_df.loc[count,'win_prob_float'])/100)
sp_plus_df['home_win_prob_sp'] = home_win_prob

# get away win probability
away_win_prob = [(1-i) for i in home_win_prob]
sp_plus_df['away_win_prob_sp'] = away_win_prob

# check probabilities equal 1
sum(sp_plus_df['away_win_prob_sp'] + sp_plus_df['home_win_prob_sp'] != 1)

0

In [46]:
# try a join on home teams find naming anomolies
test_merge_df = pd.merge(betting_df, sp_plus_df, left_on='away_team', right_on='away_team_sp', how='left')
test_merge_df[test_merge_df['away_team_sp'].isnull()]['away_team'].to_list()

['South Florida', 'UT San Antonio', "Hawai'i"]

In [47]:
# try a join on away teams find naming anomolies
test_merge_df = pd.merge(betting_df, sp_plus_df, left_on='home_team', right_on='home_team_sp', how='left')
test_merge_df[test_merge_df['home_team_sp'].isnull()]['home_team'].to_list()

['Southern Mississippi',
 'Louisiana Monroe',
 'Louisiana',
 'Miami',
 'San José State']

In [48]:
# make dict for naming conventions
sp_rename_dict = {'Miami-FL':'Miami',
                 'Southern Miss':'Southern Mississippi',
                 'UL-Monroe':'Louisiana Monroe',
                 'UL-Lafayette':'Louisiana',
                 'San Jose State':'San José State',
                 'Massachusetts':'UMass',
                 'USF':'South Florida',
                 'UTSA':'UT San Antonio',
                 'Hawaii':'Hawai\'i'}

In [49]:
# rename home teams from SP+
new_home_name_list = []
for i in sp_plus_df['home_team_sp']:
    if i in sp_rename_dict:
        new_home_name_list.append(sp_rename_dict[i])
    else:
        new_home_name_list.append(i)

# rename home teams from SP+
new_away_name_list = []
for i in sp_plus_df['away_team_sp']:
    if i in sp_rename_dict:
        new_away_name_list.append(sp_rename_dict[i])
    else:
        new_away_name_list.append(i)

In [50]:
sp_plus_df['away_team_sp_v2'] = new_away_name_list
sp_plus_df['home_team_sp_v2'] = new_home_name_list

In [51]:
# get certain columns for joining
sp_cols = ['Game','away_team_sp_v2','home_team_sp_v2','home_win_prob_sp','away_win_prob_sp','Proj. margin','Proj. winner']

In [52]:
test_merge_df = pd.merge(betting_df, sp_plus_df, left_on='away_team', right_on='away_team_sp_v2', how='left')

In [53]:
# test if there are any naming anomolies left
len(test_merge_df[test_merge_df['away_team_sp'].isnull()]['away_team'].to_list())
len(test_merge_df[test_merge_df['home_team_sp'].isnull()]['home_team'].to_list())

0

In [54]:
betting_df = pd.merge(betting_df, sp_plus_df, left_on='away_team', right_on='away_team_sp_v2', how='left')

In [55]:
# drop duplicates
betting_df = betting_df.drop_duplicates(subset=['home_team', 'away_team'])

# export to csv
betting_df.to_csv('./output/'+ df_name_prefix  + 'raw.csv')

### Home Teams - Get teams to bet on if both ESPN and SP+ have good win probs for home team

Here want to see home win probabilities for ESPN and SP+ to be more than 5% over the implied win prob % and the spread to be less than 7 points for favored away team

In [56]:
# where win prob for espn and sp greater than caesars implied win% by more than 5%
# game is close and is a home underdog

display_cols = ['home_team','home_moneyline_final', 'home_spread_final', 'Proj. margin', 'home_win_prob_no_vig_final', 'home_win_prob_espn',\
                'home_win_prob_sp', 'home_elo_prob', 'win_loss_home','home_win_%', 'ap_rank_home','coaches_rank_home','away_team','away_win_prob_no_vig_final', \
                'away_win_prob_espn', 'away_win_prob_sp', 'away_elo_prob', 'win_loss_away', 'away_win_%','ap_rank_away', 'coaches_rank_away', 'start_time']

# spread to be under for chance to win
spread_limit = 7

# percentage difference
perc_limit = 0

home_bets_df = betting_df[(betting_df['home_moneyline_final'] >= -110) &\
                          (betting_df['home_spread_final']<spread_limit)&\
                          # but need spread to be greater than 0 so is still underdog
                          (betting_df['home_spread_final']>=0)&\
                          # win probabilities for ESPN and SP+ have to have to be greater than defined 
                          # (e.g. needs to be 5% more win probability than Caesars win prob
                          ((betting_df['home_win_prob_espn'] - betting_df['home_win_prob_no_vig_final'] > perc_limit) & \
                           (betting_df['home_win_prob_sp'] - betting_df['home_win_prob_no_vig_final']> perc_limit))&\
                          # at least one of the three win probabilities are greater than 50%
                          ((betting_df['home_win_prob_espn']>.5)|\
                           (betting_df['home_win_prob_sp']>.5)|\
                           (betting_df['home_elo_prob']>.5))][display_cols]

home_bets_df

,home_team,home_moneyline_final,home_spread_final,Proj. margin,home_win_prob_no_vig_final,home_win_prob_espn,home_win_prob_sp,home_elo_prob,win_loss_home,home_win_%,...,away_team,away_win_prob_no_vig_final,away_win_prob_espn,away_win_prob_sp,away_elo_prob,win_loss_away,away_win_%,ap_rank_away,coaches_rank_away,start_time
24,UAB,-110.0,0.0,1.6,0.500000,0.620,0.54,0.574280,4-4-0,100.0,...,UT San Antonio,0.500000,0.380,0.46,0.425720,6-2-0,75.00,,,12:30:00
35,Louisiana,150.0,4.0,0.1,0.384509,0.442,0.50,0.673776,4-4-0,75.0,...,Troy,0.615491,0.558,0.50,0.326224,6-2-0,50.00,,,14:00:00
51,NC State,158.0,4.0,0.4,0.371700,0.588,0.51,0.378703,6-2-0,100.0,...,Wake Forest,0.628300,0.412,0.49,0.621297,6-2-0,66.67,20.0,19.0,17:00:00


### Away Teams - Both ESPN and SP+ have good win probs. Either ESPN or SP+ to be greater than 50%

In [57]:
# get games where the away team is not favored and have a greater than 47% chance at winning for espn and sp+
display_cols = ['away_moneyline_final', 'away_team','away_win_prob_no_vig_final', 'away_win_prob_espn', \
                'away_win_prob_sp', 'away_elo_prob', 'win_loss_away', 'away_win_%',\
                'home_team','home_moneyline_final', 'home_spread_final', 'Proj. margin', 'home_win_prob_no_vig_final', 'home_win_prob_espn',\
                'home_win_prob_sp', 'home_elo_prob', 'win_loss_home','home_win_%', 'start_time']
                
# win perc floor
win_perc_floor = .5

away_bets_df = betting_df[(betting_df['away_moneyline_final'] > 0) &\
           ((betting_df['away_win_prob_espn']>win_perc_floor)|\
           (betting_df['away_win_prob_sp']>win_perc_floor))][display_cols]
away_bets_df

,away_moneyline_final,away_team,away_win_prob_no_vig_final,away_win_prob_espn,away_win_prob_sp,away_elo_prob,win_loss_away,away_win_%,home_team,home_moneyline_final,home_spread_final,Proj. margin,home_win_prob_no_vig_final,home_win_prob_espn,home_win_prob_sp,home_elo_prob,win_loss_home,home_win_%,start_time
14,140.0,Baylor,0.400908,0.525,0.40,0.706975,5-3-0,50.00,Oklahoma,-165.0,-3.5,4.5,0.599092,0.475,0.60,0.293025,5-3-0,75.00,12:00:00
23,100.0,Oklahoma State,0.478261,0.542,0.66,0.840723,6-2-0,33.33,Kansas,-120.0,-1.0,7.1,0.521739,0.458,0.34,0.159277,5-3-0,75.00,12:30:00
25,158.0,Syracuse,0.371700,0.545,0.47,0.475555,6-2-0,50.00,Pittsburgh,-190.0,-4.0,1.1,0.628300,0.455,0.53,0.524445,4-4-0,60.00,12:30:00
34,110.0,Texas State,0.457256,0.525,0.60,0.710540,3-5-0,0.00,Louisiana Monroe,-130.0,-1.5,4.6,0.542744,0.475,0.40,0.289460,2-6-0,66.67,14:00:00


### Merged pick list

#### Wrangle and export data to csv

In [74]:
betting_df.head()

,id,season,week,season_type,start_date,home_team,home_conference,home_score,away_team,away_conference,...,O/U pick,O/U diff,Game_v2,away_team_sp,home_team_sp,win_prob_float,home_win_prob_sp,away_win_prob_sp,away_team_sp_v2,home_team_sp_v2
0,401404141,2022,10,regular,2022-11-05 08:30:00+00:00,Army,FBS Independents,,Air Force,Mountain West,...,Over,15.6,"[Air Force, Army]",Air Force,Army,66,0.34,0.66,Air Force,Army
1,401415649,2022,10,regular,2022-11-05 09:00:00+00:00,Tulsa,American Athletic,,Tulane,American Athletic,...,Over,0.4,"[Tulane, Tulsa]",Tulane,Tulsa,66,0.34,0.66,Tulane,Tulsa
2,401405129,2022,10,regular,2022-11-05 09:00:00+00:00,Northwestern,Big Ten,,Ohio State,Big Ten,...,Under,-7.5,"[Ohio State, Northwestern]",Ohio State,Northwestern,99,0.01,0.99,Ohio State,Northwestern
3,401405128,2022,10,regular,2022-11-05 09:00:00+00:00,Nebraska,Big Ten,,Minnesota,Big Ten,...,Over,4.1,"[Minnesota, Nebraska]",Minnesota,Nebraska,72,0.28,0.72,Minnesota,Nebraska
4,401405130,2022,10,regular,2022-11-05 09:00:00+00:00,Purdue,Big Ten,,Iowa,Big Ten,...,Over,3.1,"[Iowa, Purdue]",Iowa,Purdue,51,0.51,0.49,Iowa,Purdue


In [58]:
away_bets_df_v2 = away_bets_df
home_bets_df_v2 = home_bets_df
away_display_cols = ['away_team', 'home_team', 'away_moneyline_final', 'home_spread_final', 'away_win_prob_no_vig_final', 'away_win_prob_espn', \
                'away_win_prob_sp', 'away_elo_prob', 'start_time']
home_display_cols = ['home_team', 'away_team', 'home_moneyline_final', 'home_spread_final', 'home_win_prob_no_vig_final', 'home_win_prob_espn', \
                'home_win_prob_sp', 'home_elo_prob', 'start_time']

In [59]:
away_bets_df_v2 = away_bets_df_v2[away_display_cols]
away_bets_df_v2 = away_bets_df_v2.rename(columns = {'away_team':'picked_team', \
                                                    'away_moneyline_final':'moneyline', \
                                                    'away_win_prob_no_vig_final':'win_prob_no_vig', \
                                                    'away_win_prob_espn':'win_prob_espn', \
                                                    'away_win_prob_sp':'sp_prob', \
                                                    'away_elo_prob':'elo_prob', \
                                                    'home_spread_final': 'spread',\
                                                    'home_team':'opponent'})

In [60]:
home_bets_df_v2 = home_bets_df_v2[home_display_cols]
home_bets_df_v2 = home_bets_df_v2.rename(columns = {'home_team':'picked_team', \
                                                    'home_moneyline_final':'moneyline', \
                                                    'home_win_prob_no_vig_final':'win_prob_no_vig', \
                                                    'home_win_prob_espn':'win_prob_espn', \
                                                    'home_win_prob_sp':'sp_prob', \
                                                    'home_elo_prob':'elo_prob', \
                                                    'home_spread_final': 'spread',\
                                                    'away_team': 'opponent'})

In [61]:
picks_df = pd.concat([home_bets_df_v2, away_bets_df_v2])

In [62]:
# export dataframe to csv
picks_df.to_csv('./output/' + df_name_prefix + '.picks.csv', index = False)

#### See pick list

In [63]:
# see new picks in dataframe legs in dataframe in this notebook
picks_df.sort_values('start_time')

,picked_team,opponent,moneyline,spread,win_prob_no_vig,win_prob_espn,sp_prob,elo_prob,start_time
14,Baylor,Oklahoma,140.0,-3.5,0.400908,0.525,0.40,0.706975,12:00:00
24,UAB,UT San Antonio,-110.0,0.0,0.500000,0.620,0.54,0.574280,12:30:00
23,Oklahoma State,Kansas,100.0,-1.0,0.478261,0.542,0.66,0.840723,12:30:00
25,Syracuse,Pittsburgh,158.0,-4.0,0.371700,0.545,0.47,0.475555,12:30:00
35,Louisiana,Troy,150.0,4.0,0.384509,0.442,0.50,0.673776,14:00:00
34,Texas State,Louisiana Monroe,110.0,-1.5,0.457256,0.525,0.60,0.710540,14:00:00
51,NC State,Wake Forest,158.0,4.0,0.371700,0.588,0.51,0.378703,17:00:00


## Parlay Maker
1) Parlay all the above into one parlay. This can be considered a "100% win bonus". Wager = \$2 (minimum for parlay)
2) Add some home favorites to parlay parlay 1. See how much the payout will increase with these heavy home favorites. Wager = \$2 (minimum for parlay)
3) Add 1 to 2 "hail mary" underdogs to parlay parlay 2. Want <b>realistic</b> underdogs, and ideally at home. These will increase payout exponentially with a minimum increased risk
Wager = \$2 (minimum for parlay)

### Parlay 1: Generate the parlay with the above picks

In [64]:
# make a parlay list of moneylines
#parlay_list = betting_df[betting_df['home_team'].isin(betting_df_v2['home_team'])]['home_moneyline'].to_list()
parlay_list = picks_df['moneyline'].to_list()
# save calculated multiplier for the parlay
multiplier = int(parlay_multiplier(parlay_list))
# save pretty list of teams
team_parlay_list = picks_df['picked_team'].to_list()


bet = 2
payout = bet*multiplier

print('Parlay 1:')
print()

print('Bet: ', "${:0,.0f}".format(bet))
print('Number of legs: ' + str(len(parlay_list)))
print('Multiplier: ', "+{:0,.0f}".format(multiplier))
print()
print('Payout: ', "${:0,.0f}".format(payout))

print()
print(team_parlay_list)

Parlay 1:

Bet:  $2
Number of legs: 7
Multiplier:  +320

Payout:  $640

['UAB', 'Louisiana', 'NC State', 'Baylor', 'Oklahoma State', 'Syracuse', 'Texas State']


### Parlay 2: Add home favorites to increase payout <font color = red>(human interaction here)</font>
Add a couple home favorites so that the payoput increases.

Made a new parlay since it's not 100% these home favorites win. If include with parlay 1, could still lose. Adding the additional payout without much at risk.

#### Explore home favorites to add to parlay 

See if there are any home favorites I feel confident in winning so that parlay payout increases. Ideally want a game where the home team is not favored by a lot according to sportsbook, but I feel is heavy favorite
- The book sees the game being close, but I see it as being a much higher win probability. Example, was Penn State @ Michigan 10/15/2022. Moneyline was -278, but I had it as a lot higher in my head

Don't need to add any if don't see anything that adds value to parlay

<b>Need to find a quantitative way to do this</b>

In [65]:
# get games where home team is favored by more than 21
# get games where the home team is not favored based on moneyline
betting_df[(betting_df['home_moneyline'] < 0) & \
           (betting_df['spread'] < -6)].sort_values('home_win_prob_no_vig_pretty', ascending = True)[display_cols];

#### Add home favorites to parlay

If there are any potential home favorites to include, then add them the the home_fav_list object. 

In [66]:
# home favorites selections
# don't need to add any if don't see anything that adds value to parlay
home_fav_list = ['Utah', 'USC', 'Arkansas'] # user input
parlay_new_odds_list = betting_df[betting_df['home_team'].isin(home_fav_list)]['home_moneyline_final'].to_list()
parlay_list = parlay_list + parlay_new_odds_list

# add to pretty list of teams
team_parlay_list = team_parlay_list + home_fav_list

multiplier = int(parlay_multiplier(parlay_list))

#### See what the new payout is

In [67]:
bet = 2
payout = bet*multiplier

print('Parlay 2:')
print()

print('Bet: ', "${:0,.0f}".format(bet))
print('Number of legs: ' + str(len(parlay_list)))
print('Multiplier: ', "+{:0,.0f}".format(multiplier))
print()
print('Odds: ', "+{:0,.0f}".format(multiplier*100))
print('Payout: ', "${:0,.0f}".format(payout))

print()
print(team_parlay_list)

Parlay 2:

Bet:  $2
Number of legs: 10
Multiplier:  +431

Odds:  +43,100
Payout:  $862

['UAB', 'Louisiana', 'NC State', 'Baylor', 'Oklahoma State', 'Syracuse', 'Texas State', 'Utah', 'USC', 'Arkansas']


### Parlay 3: Add 1 to 2 "Hail Mary" picks

Attempting the chance at a lotto parlay (very high payout on $2 bet). Do this by adding 1 to 2 underdogs so the payout increases.

Want to see all of them, but sort by moneyline odds.

Possible to select on "after dark games" for the opportunity to hedge?

#### Home Underdogs
Get home underdogs, but exclude those included in parlay 1

In [68]:
betting_df[(~betting_df['home_team'].isin(team_parlay_list)) & (betting_df['home_moneyline_final']>0)][display_cols].sort_values('home_moneyline_final');

In [69]:
# home hail mary selections
# don't need to add any here if will add in hail mary away list below
hail_mary_home_list = ['Notre Dame'] 

if len(hail_mary_home_list) == 0:
    parlay_list = parlay_list
    team_parlay_list = team_parlay_list
elif len(hail_mary_home_list) > 0:
    parlay_new_odds_list = betting_df[betting_df['home_team'].isin(hail_mary_home_list)]['home_moneyline_final'].to_list()
    parlay_list = parlay_list + parlay_new_odds_list
    team_parlay_list = team_parlay_list + hail_mary_home_list

#### Away Underdogs

In [70]:
# colummns to see in this dataframe
display_cols = ['start_time','outlet', 'home_team','home_moneyline_final', 'home_win_prob_no_vig_final', \
                'win_loss_home','home_win_%', 'ap_rank_home','coaches_rank_home',\
                'away_team','away_moneyline_final','away_win_prob_no_vig_final', 'win_loss_away', 'away_win_%','ap_rank_away', 'coaches_rank_away',\
                'formatted_spread']
betting_df[(betting_df['away_moneyline_final']>0)][display_cols].sort_values('away_moneyline_final');

In [71]:
# away hail mary selections
# don't need to add any here if added in hail mary home list
hail_mary_away_list = ['Tennessee']

if len(hail_mary_away_list) == 0:
    parlay_list = parlay_list
    team_parlay_list = team_parlay_list
elif len(hail_mary_away_list) > 0:
    parlay_new_odds_list = betting_df[betting_df['away_team'].isin(hail_mary_away_list)]['away_moneyline_final'].to_list()
    parlay_list = parlay_list + parlay_new_odds_list
    team_parlay_list = team_parlay_list + hail_mary_away_list

multiplier = int(parlay_multiplier(parlay_list))

#### Get new parlay payout

In [72]:
bet = 2
payout = bet*multiplier

print('Parlay 3:')
print()

print('Bet: ', "${:0,.0f}".format(bet))
print('Number of legs: ' + str(len(parlay_list)))
print('Multiplier: ', "+{:0,.0f}".format(multiplier))
print()
print('Odds: ', "+{:0,.0f}".format(multiplier*100))
print('Payout: ', "${:0,.0f}".format(payout))

print()
print(team_parlay_list)
print()

display_cols = ['start_time','outlet', 'home_team', 'home_moneyline_final', 'home_win_prob_no_vig_final', \
                'away_team', 'away_moneyline_final','away_win_prob_no_vig_final']
# output final parlay to csv
betting_df[(betting_df['home_team'].isin(team_parlay_list)) | (betting_df['away_team'].isin(team_parlay_list))]\
    [display_cols].to_csv('./output/' + df_name_prefix + '.parlay_final.csv', index = False)
betting_df[(betting_df['home_team'].isin(team_parlay_list)) | (betting_df['away_team'].isin(team_parlay_list))][display_cols];

Parlay 3:

Bet:  $2
Number of legs: 12
Multiplier:  +3,693

Odds:  +369,300
Payout:  $7,386

['UAB', 'Louisiana', 'NC State', 'Baylor', 'Oklahoma State', 'Syracuse', 'Texas State', 'Utah', 'USC', 'Arkansas', 'Notre Dame', 'Tennessee']



# Notes from trying to back test

1. Records are not in line with week tested. They are only up to current time. If back test need to create new win loss



#### Get data from ESPN's from pick center

In [ ]:
import requests
session = requests.Session()

# Create the payload
payload = {'email':'re4693go@gmail.com',
          'password':'BNg3!@^j4^wz'
         }

# Post the payload to the site to log in
s = session.post("https://www.espn.com/login/?returnURL=https://www.espn.com/insider/pickcenter", data=payload)

# Navigate to the next page and scrape the data
s = session.get('https://insider.espn.com/insider/pickcenter/ncf/game?gameid=401404103')

In [ ]:
link = 'https://insider.espn.com/insider/pickcenter/ncf/game?gameid=401404103'

In [ ]:
page = requests.get(link)
soup = BeautifulSoup(page.content, "html.parser")

In [ ]:
soup.find_all('div', class_='percent')

#### Validate # of games on espn website vs dataframe

In [ ]:
# check length of dataframe to see if it includes everything
# compare against scraped espn scoreboard
len(betting_df)

In [ ]:
# 'https://www.espn.com/college-football/schedule/_/week/10/year/2022/seasontype/2'
espn_game_num = 

In [ ]:
'https://www.espn.com/college-football/schedule/_/week/10/year/2022/seasontype/2'
scoreboard_url = 'https://www.espn.com/college-football/schedule/_/week/' + str(week_num) + \
    '/year/' + str(year_num) + '/seasontype/2'
page = requests.get(scoreboard_url)
soup = BeautifulSoup(page.content, "html.parser")


# use the beautiful soup object to parse
#results = soup.find(id='gamepackage-pick-center')
# find home team data and add to lists
# home team data
#home_data = results.find_all('tr', class_='hometeam')

In [ ]:
day_data = soup.find_all('tr', class_='Table__TR Table__TR--sm Table__even')
for i in day_data:
    print(i)
    print()
